In [107]:
# Upload dataset
from google.colab import files
uploaded = files.upload()

Saving data-iris.csv to data-iris (1).csv
Saving data-iris-values.csv to data-iris-values (1).csv


In [0]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# %matplotlib inline

In [109]:
print("Versions:")
print("Tensorflow:", tf.VERSION)
# 1.14.0-rc1
print("Keras: ", tf.keras.__version__)
#2.2.4-tf

Versions:
Tensorflow: 1.14.0-rc1
Keras:  2.2.4-tf


In [0]:
# Fix random seed for reproducibility
numpy.random.seed(7)

In [0]:
# Load iris dataset
path = "./data-iris.csv"
dataset = pd.read_csv(path)

In [112]:
features = list(dataset.columns.values)
print(features)

['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species']


In [0]:
features.remove('Species')
features.remove('Id')
X = dataset[features]
y = dataset['Species']

In [0]:
TESTSIZE = 0.25
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TESTSIZE, random_state=0)

In [0]:
# Ensure that fieldnames aren't included
X_train = X_train.values
y_train = y_train.values
X_test  = X_test.values
y_test  = y_test.values

In [0]:
NB_EPOCHS = 500  # num of epochs to test for
BATCH_SIZE = 8

In [0]:
## Create our model
model = tf.keras.Sequential()
model.add(layers.Dense(6, input_dim=4, activation='elu'))
model.add(layers.Dense(6, activation='elu'))
model.add(layers.Dense(3, activation='sigmoid' ))

In [0]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
# checkpoint: store the best model
ckpt_model = 'iris-weights.best.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(ckpt_model,
                            monitor='val_acc',
                            verbose=1,
                            save_best_only=True,
                            mode='max')
callbacks_list = [checkpoint]

In [148]:
# Train the model, store the results for plotting
print('Training...')
history = model.fit(X_train,
                    y_train,
                    validation_data=(X_test, y_test),
                    nb_epoch=NB_EPOCHS,
                    batch_size=BATCH_SIZE,
                    callbacks=callbacks_list,
                    verbose=0)

W0619 23:07:43.988224 140111316580224 training.py:593] The `nb_epoch` argument in `fit` has been renamed `epochs`.


Training...


ValueError: ignored

In [0]:
# Model accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.show()

In [0]:
# Model Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.show()

In [0]:
# load model
model = tf.keras.models.load_model('iris-weights.best.hdf5')
# summarize model.
model.summary()
# load dataset
dataset = numpy.loadtxt("data-iris-values.csv", delimiter=",")

In [0]:
# split into input (X) and output (Y) variables
X = dataset[:,1:5]
Y = dataset[:,5]

# evaluate the model
score = model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
